In [1]:
%pip install psycopg2
%pip install nbformat

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: C:\Users\gogco\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: C:\Users\gogco\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
import nbformat
import psycopg2 as pg
import pandas as pd

### Введите данные для подключения к существующей или создание новой БД

In [ ]:
DB_NAME = "hw2"
DB_USER = "postgres"
DB_PASSWORD = ""
DB_HOST = "localhost"
DB_PORT = "5433"

In [4]:
# подключение к системной БД
conn = pg.connect(
    dbname="postgres",
    user=DB_USER,
    password=DB_PASSWORD,
    host=DB_HOST,
    port=DB_PORT
)
conn.autocommit = True
cur = conn.cursor()

In [5]:
# попытка создания БД с указанным именем
cur.execute(f"SELECT 1 FROM pg_database WHERE datname = '{DB_NAME}';")
if not cur.fetchone():
    cur.execute(f"create database {DB_NAME} with owner = {DB_USER};")
    print(f"База данных {DB_NAME} успешно создана!")
else:
    print(f"База данных {DB_NAME} уже существует.")
cur.close()
conn.close()

База данных hw2 уже существует.


In [7]:
# подключение к существующей или созданной БД
conn = pg.connect(
    dbname=DB_NAME,
    user=DB_USER,
    password=DB_PASSWORD,
    host=DB_HOST,
    port=DB_PORT
)
cur = conn.cursor()

### Создание таблиц и отношений БД SQL-запросом

In [9]:
try:
    with open("schema.sql", "r", encoding="utf-8") as f:
        sql_script = f.read()
    cur.execute(sql_script)
    conn.commit()

    # проверка, создались ли таблицы
    cur.execute("SELECT table_name FROM information_schema.tables WHERE table_schema = 'public';")
    tables = cur.fetchall()

    if tables:
        print("Таблицы успешно созданы!")
    else:
        print("Ошибка: таблицы не были созданы!")

except Exception as e:
    print(f"Ошибка при создании таблиц: {e}")
    conn.rollback()  # откат изменений, если была ошибка

Таблицы успешно созданы!


In [10]:
# функция изменяет разделитель в CSV-файле
def change_csv_delimiter(input_file, output_file=None, old_delimiter=';', new_delimiter=','):
    df = pd.read_csv(input_file, delimiter=old_delimiter)
    if output_file is None:
        output_file = input_file
    df.to_csv(output_file, sep=new_delimiter, index=False)
    
# функция изменяет разделитель чисел с плавающей точкой
def change_format_numbers(series: pd.Series, old_delimiter=',', new_delimiter='.'):
    return pd.to_numeric(series.astype(str).str.replace(old_delimiter, new_delimiter, regex=False), errors='coerce')

# функция формат данных в CSV-файле
def change_format_data(series: pd.Series, old_format='%d.%m.%Y', new_format='%Y-%m-%d'):
    return pd.to_datetime(series, format=old_format).dt.strftime(new_format)

In [ ]:
change_csv_delimiter('data/customer.csv')
change_csv_delimiter('data/transaction.csv')

In [ ]:
transaction_df = pd.read_csv("data/transaction.csv", delimiter=",")
transaction_df["standard_cost"] = change_format_numbers(transaction_df["standard_cost"])
transaction_df["list_price"] = change_format_numbers(transaction_df["list_price"])
transaction_df["transaction_date"] = change_format_data(transaction_df["transaction_date"])
transaction_df = transaction_df[transaction_df["customer_id"] <= 4000] # важно! выполнено удаление всех транзакций, которые имеют невалидный customer_id 
                                                                       #(вероятно, изначально таблица customer имела больше записей, 
                                                                       # а таблица transaction хранит транзакции уже несуществующих клиентов)
transaction_df.to_csv("data/transaction.csv", index = False)

In [18]:
# функция загрузки CSV в PostgreSQL
def load_csv_to_postgres(cur, csv_path, table_name):
    df = pd.read_csv(csv_path)
    # открытие файла и загрузка в PostgreSQL
    with open(csv_path, 'r', encoding='utf-8') as f:
        next(f)  # пропуск заголовков
        try:
            cur.copy_from(f, table_name, sep=',', null='')
            conn.commit()
            print(f"Данные из {csv_path} загружены в {table_name}")
        except Exception as e:
            conn.rollback()
            print(f"Ошибка при загрузке {csv_path} в {table_name}: {e}")

# функция выполняет SQL-запросы
def execute_sql_file(file_path, conn):
    try: 
        # читает SQL-файл
        with open(file_path, 'r', encoding='utf-8') as sql_file:
            sql_script = sql_file.read()
        
        # выполняет SQL-запрос
        # cur.execute(sql_script)
        # conn.commit()  # фиксирует изменения
        
        df = pd.read_sql_query(sql_script, conn)
        print(f"SQL-файл {file_path} выполнен успешно.")
        return df
    
    except Exception as e:
        conn.rollback()  # отменяет изменения при ошибке
        print(f"Ошибка при выполнении {file_path}: {e}")

In [14]:
# загрузка данных в таблицы (порядок загрузки индивидуален в соответствии со схемой)
load_csv_to_postgres(cur, "data/customer.csv", "customer")
load_csv_to_postgres(cur, "data/transaction.csv", "transaction")

Данные из data/customer.csv загружены в customer
Данные из data/transaction.csv загружены в transaction


In [19]:
execute_sql_file("sql/1.sql", conn)

SQL-файл sql/1.sql выполнен успешно.


C:\Users\gogco\AppData\Local\Temp\ipykernel_14444\3059630634.py:26: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql_script, conn)


,brand
0,Solex
1,Giant Bicycles
2,OHM Cycles
3,Trek Bicycles


In [20]:
execute_sql_file("sql/2.sql", conn)

SQL-файл sql/2.sql выполнен успешно.


C:\Users\gogco\AppData\Local\Temp\ipykernel_14444\3059630634.py:26: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql_script, conn)


,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost
0,17,79,2426,2017-04-03,False,Approved,Norco Bicycles,Standard,medium,medium,1555.58,818.01
1,19,54,2268,2017-04-06,True,Approved,WeareA2B,Standard,medium,medium,1292.84,13.44
2,23,37,2001,2017-04-08,True,Approved,OHM Cycles,Standard,low,medium,1793.43,248.82
3,83,0,3398,2017-04-01,True,Approved,OHM Cycles,Standard,medium,medium,235.63,125.07
4,89,0,2682,2017-04-04,True,Approved,OHM Cycles,Road,high,large,12.01,7.21
...,...,...,...,...,...,...,...,...,...,...,...,...
526,19655,0,336,2017-04-09,True,Approved,Norco Bicycles,Standard,medium,medium,360.40,270.30
527,19853,7,3072,2017-04-02,False,Approved,Trek Bicycles,Road,low,medium,980.37,234.43
528,19899,57,325,2017-04-06,False,Approved,WeareA2B,Touring,medium,large,1890.39,260.14
529,19968,0,2751,2017-04-06,False,Approved,WeareA2B,Standard,medium,medium,60.34,45.26


In [21]:
execute_sql_file("sql/3.sql", conn)

SQL-файл sql/3.sql выполнен успешно.


C:\Users\gogco\AppData\Local\Temp\ipykernel_14444\3059630634.py:26: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql_script, conn)


,job_title
0,Senior Cost Accountant
1,Senior Developer
2,Senior Editor
3,Senior Financial Analyst
4,Senior Quality Engineer
5,Senior Sales Associate


In [22]:
execute_sql_file("sql/4.sql", conn)

SQL-файл sql/4.sql выполнен успешно.


C:\Users\gogco\AppData\Local\Temp\ipykernel_14444\3059630634.py:26: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql_script, conn)


,brand
0,None
1,WeareA2B
2,Norco Bicycles
3,Solex
4,Giant Bicycles
5,OHM Cycles
6,Trek Bicycles


In [23]:
execute_sql_file("sql/5.sql", conn)

SQL-файл sql/5.sql выполнен успешно.


C:\Users\gogco\AppData\Local\Temp\ipykernel_14444\3059630634.py:26: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql_script, conn)


,customer_id,first_name,last_name,gender,dob,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,address,postcode,state,country,property_valuation
0,1,Laraine,Medendorp,F,1953-10-12,Executive Secretary,Health,Mass Customer,N,Yes,060 Morning Avenue,2016,New South Wales,Australia,10
1,2,Eli,Bockman,Male,1980-12-16,Administrative Officer,Financial Services,Mass Customer,N,Yes,6 Meadow Vale Court,2153,New South Wales,Australia,10
2,3,Arlin,Dearle,Male,1954-01-20,Recruiting Manager,Property,Mass Customer,N,Yes,0 Holy Cross Court,4211,QLD,Australia,9
3,5,Sheila-kathryn,Calton,Female,1977-05-13,Senior Editor,None,Affluent Customer,N,Yes,9 Oakridge Court,3216,VIC,Australia,9
4,6,Curr,Duckhouse,Male,1966-09-16,None,Retail,High Net Worth,N,Yes,4 Delaware Trail,2210,New South Wales,Australia,9
5,7,Fina,Merali,Female,1976-02-23,None,Financial Services,Affluent Customer,N,Yes,49 Londonderry Lane,2650,New South Wales,Australia,4
6,8,Rod,Inder,Male,1962-03-30,Media Manager I,None,Mass Customer,N,No,97736 7th Trail,2023,New South Wales,Australia,12
7,11,Uriah,Bisatt,Male,1954-04-30,None,Property,Mass Customer,N,No,2 Sutherland Street,3799,VIC,Australia,6
8,12,Sawyere,Flattman,Male,1994-07-21,Nuclear Power Engineer,Manufacturing,Mass Customer,N,No,9 Mcbride Trail,2760,New South Wales,Australia,8
9,13,Gabriele,Norcross,Male,1955-02-15,Developer I,Financial Services,High Net Worth,N,Yes,9861 New Castle Avenue,2428,New South Wales,Australia,9


In [24]:
execute_sql_file("sql/6.sql", conn)

SQL-файл sql/6.sql выполнен успешно.


C:\Users\gogco\AppData\Local\Temp\ipykernel_14444\3059630634.py:26: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql_script, conn)


,customer_id,first_name,last_name,gender,dob,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,address,postcode,state,country,property_valuation
0,3558,Torey,Sarney,Male,1954-09-05,None,None,High Net Worth,N,Yes,4301 Haas Junction,2204,NSW,Australia,9
1,852,Andie,Bonney,Female,2000-11-04,Compensation Analyst,Financial Services,Affluent Customer,N,Yes,94 Anhalt Way,3139,VIC,Australia,7
2,869,Addia,Abels,Female,1984-03-11,Account Representative I,Financial Services,High Net Worth,N,Yes,02377 Maywood Trail,2287,NSW,Australia,7
3,1373,Shaylynn,Epsley,Female,1958-09-23,Director of Sales,Financial Services,Mass Customer,N,Yes,0 Grasskamp Pass,3170,VIC,Australia,10
4,2074,Roslyn,Rawdall,Female,1997-06-11,None,Financial Services,Mass Customer,N,No,95483 Washington Junction,2505,NSW,Australia,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502,3996,Rosalia,Halgarth,Female,1975-08-09,VP Product Management,Health,Mass Customer,N,No,57042 Village Green Point,4511,QLD,Australia,6
503,3997,Blanch,Nisuis,Female,2001-07-13,Statistician II,Manufacturing,High Net Worth,N,Yes,87 Crescent Oaks Alley,2756,NSW,Australia,10
504,3998,Sarene,Woolley,U,None,Assistant Manager,IT,High Net Worth,N,No,8194 Lien Street,4032,QLD,Australia,7
505,3999,Patrizius,None,Male,1973-10-24,None,Manufacturing,Affluent Customer,N,Yes,320 Acker Drive,2251,NSW,Australia,7


In [25]:
execute_sql_file("sql/7.sql", conn)

SQL-файл sql/7.sql выполнен успешно.


C:\Users\gogco\AppData\Local\Temp\ipykernel_14444\3059630634.py:26: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql_script, conn)


,customer_id,first_name,last_name,gender,dob,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,address,postcode,state,country,property_valuation
0,3473,Sanderson,Alloway,U,None,Analog Circuit Design manager,IT,Mass Customer,N,No,3 Roxbury Street,2261,NSW,Australia,7
1,893,Gibby,Fearnley,Male,1983-09-11,Geologist I,IT,Mass Customer,N,No,6382 Bayside Street,2153,NSW,Australia,10
2,3151,Thorn,Choffin,U,None,Senior Developer,IT,Affluent Customer,N,Yes,5323 Chive Avenue,2486,NSW,Australia,7
3,34,Jephthah,Bachmann,U,1843-12-21,Legal Assistant,IT,Affluent Customer,N,No,90 Lawn Parkway,4805,QLD,Australia,4
4,2913,Padraic,Bonnar,Male,1955-07-11,VP Quality Control,IT,Affluent Customer,N,Yes,937 Sloan Center,4118,QLD,Australia,6
5,1918,Devin,Sandeson,U,None,Staff Accountant II,IT,Affluent Customer,N,No,51763 Carey Place,3056,VIC,Australia,7
6,1672,Sharla,Creebo,Female,1963-04-27,Design Engineer,IT,Affluent Customer,N,Yes,1 Morning Circle,2250,NSW,Australia,8
7,975,Goldarina,Rzehorz,U,None,Automation Specialist IV,IT,Mass Customer,N,No,938 Monica Park,3173,VIC,Australia,9
8,1773,Nickolas,Guittet,U,None,None,IT,Mass Customer,N,Yes,835 West Hill,2118,NSW,Australia,11


In [26]:
execute_sql_file("sql/8.sql", conn)

SQL-файл sql/8.sql выполнен успешно.


C:\Users\gogco\AppData\Local\Temp\ipykernel_14444\3059630634.py:26: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql_script, conn)


,customer_id,first_name,last_name,gender,dob,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,address,postcode,state,country,property_valuation
0,22,Deeanne,Durtnell,Female,1962-12-10,None,IT,Mass Customer,N,No,52 Carey Alley,4740,QLD,Australia,5
1,28,Fee,Zellmer,Male,1973-09-30,Senior Quality Engineer,Health,Affluent Customer,N,Yes,2951 Petterle Place,2756,New South Wales,Australia,9
2,41,Basilius,Coupe,Male,1976-04-14,Food Chemist,Health,Mass Customer,N,No,2028 Lakewood Place,2480,New South Wales,Australia,3
3,47,Matthew,Jeaycock,Male,1992-10-22,Registered Nurse,Health,Affluent Customer,N,No,4853 Gulseth Avenue,4503,QLD,Australia,5
4,104,Odille,Panketh,Female,1978-06-06,Automation Specialist II,Health,Mass Customer,N,Yes,0410 Division Junction,2750,New South Wales,Australia,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,3255,Sutherlan,Truin,U,None,Engineer IV,IT,High Net Worth,N,No,12357 Arapahoe Parkway,4740,QLD,Australia,3
111,3288,Fair,Dewen,U,None,Engineer III,IT,High Net Worth,N,No,6 Golf Center,2042,NSW,Australia,10
112,3360,Joelie,Sherlaw,Female,1963-03-23,Quality Engineer,Health,Affluent Customer,N,No,62551 Del Mar Avenue,2300,NSW,Australia,6
113,3365,Karlens,Chaffyn,U,None,Engineer III,IT,Mass Customer,N,No,7234 Dawn Alley,4670,QLD,Australia,1


In [27]:
cur.close()
conn.close()